In [4]:
# データの読み込み
import pandas as pd
import numpy as np
from collections import Counter
import math
from scipy.optimize import minimize
import os
import glob
import openpyxl
import math
from scipy.optimize import minimize, fsolve
from scipy import optimize, exp
import scipy.optimize as optimize
from sympy import *
import sympy

r = 1.00

# NumPyの表示オプションを設定して、整数表記にする
np.set_printoptions(suppress=True, precision=4)

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# ディレクトリ内のxlsxファイルのリストを取得 ###路線名変更
directory_path = '/Users/takuyamiyajima/Python/研究室/JFK-LAX'  # ディレクトリのパスを指定
xlsx_files = [f for f in os.listdir(directory_path) if f.endswith('.xlsx')]

# xlsxファイルを読み込んでDataFrameに格納し、リストに追加
dataframes = []
for xlsx_file in xlsx_files:
    file_path = os.path.join(directory_path, xlsx_file)
    df = pd.read_excel(file_path, index_col=0, engine="openpyxl")
    # 行名に"AA"を含む行のみを抽出 ###航空会社名変更
    df = df[df.index.astype(str).str.contains('AA', na=False)]
    dataframes.append(df)

# DataFrameを結合（インデックスをリセット）
df = pd.concat([df.reset_index(drop=True) for df in dataframes], axis=0) 

### クラス名変更
# Main Cabinの価格のみ取り出す
df = df.reset_index(drop=True)
def process_column(column):
    index_to_keep = []
    previous_row = None

    for index, cell in enumerate(column):
        if cell == 'Main Cabin':
            if previous_row is not None:
                index_to_keep.append(previous_row)
        previous_row = index
    
    return column.loc[index_to_keep]
df = df.apply(process_column)
# すべての列をfloat型に変換
df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))

# DataFrameをNumPyの配列に変換
np_array = df.values

# 重複を除いた値を抽出してリストに格納する
unique_values = np.unique(np_array)
unique_values_list = unique_values.tolist()
# nanを削除
unique_values_list = [int(x) for x in unique_values_list if not np.isnan(x)]
print(unique_values_list)

[168, 173, 178, 188, 194, 198, 199, 203, 204, 208, 213, 218, 219, 223, 224, 228, 229, 234, 244, 249, 254, 259, 263, 264, 269, 274, 284, 293, 294, 299, 303, 304, 309, 313, 314, 324, 344, 349, 354, 359, 369, 374, 379, 383, 384, 389, 394, 422, 429, 431, 432, 439, 442, 449, 452, 459, 462, 464, 473, 474, 479, 558, 559, 569, 574, 579, 584, 589, 598, 599, 609, 674, 683, 684, 694, 704, 734, 744, 754, 809, 819, 829, 884, 889, 894, 899, 909, 919, 929, 939, 949, 959, 999, 1019, 1029, 1039, 1054, 1059, 1615]


In [5]:
# 満期n日の場合
def pricing(data, target):
    gamma = 0
    r = 1
    K = target * r
    # 初期推定値を設定
    initial_guess = 0.0  # 初期推定値
    lower_bound = 0
    upper_bound = 1
    # 制約条件を定義
    constraint = ({'type': 'ineq', 'fun': lambda beta: beta - lower_bound},  # beta >= lower_bound
                {'type': 'ineq', 'fun': lambda beta: upper_bound - beta})  # beta <= upper_bound

    # 最小化したい関数を定義
    def objective_function1(beta):
        term = 0
        for i in range(len(data)):
            if data[i,0] - target != 0:
                term += (1/(1-gamma)) * data[i,1] * (( data[i,0] - target ) * beta)**(1-gamma)
            else:
                term += 0
        return term
        # return np.exp(2 * beta) + np.exp(-2 * beta)

    def objective_function2(beta):
        term = 0
        for i in range(len(data)):
            if data[i,0] != target:
                term += (1/(1-gamma)) * data[i,1] * (( data[i,0] - target ) * beta + max([ data[i,0] - target , 0 ]))**(1-gamma)
            else:
                term += 0
        return term
        # return math.exp(2 * beta) + math.exp(-2 * beta -2)

    # 最大化を実行
    result = minimize(objective_function1, initial_guess, constraints=constraint)
    # 最適な変数の値を表示
    beta1 = result.x[0]
    # print(beta1)
    # print(result.fun)

    # 最大化を実行
    result = minimize(objective_function2, initial_guess, constraints=constraint)
    # 最適な変数の値を表示
    beta2 = result.x[0]
    # print(beta2)
    # print(result.fun)

    x = sympy.Symbol('x')
    v = sympy.Symbol('v')

    A = 0
    for i in range(len(data)):
        A += - (1/(1-gamma)) * abs(data[i,1]) * ( x + ( data[i,0] - target ) * beta1 ) **(1-gamma)
    B = 0
    for i in range(len(data)):
        B += - (1/(1-gamma)) * abs(data[i,1]) * ( (x-v) + ( data[i,0] - target ) * beta2 + abs(max([ data[i,0] - target , 0 ])))**(1-gamma)

    # solutions = solve([A, B], [x, v])
    # print(solutions)

    solutions = solve(A-B, v)
    for sol in solutions:
        if sol.is_real:
            return sol

    # # 数値的な解を求める
    # try:
    #     solutions = nsolve([A, B], [x, v], [100,100])
    #     # v の実数解を出力
    #     v_solution = solutions[0][1]
    #     print(f"v_solution = {v_solution}")
    #     return v_solution
    
    # except ZeroDivisionError:
    #     print("ZeroDivisionError")

# 翌日のツリーを作成
def tree(target_value):
    if np.any(np_array == target_value):
        # 条件に一致する位置を取得
        indices = np.where(np_array == target_value)

        # 一致する位置の行と列をndarrayに格納
        positions = np.array(list(zip(indices[0], indices[1])))

        # 列番号を1足し、その場所の値をリストに格納
        values = []
        for row, col in positions:
            if col != np_array.shape[1] - 1:
                values.append(np_array[row, col + 1])

        # nanを削除
        values = [x for x in values if not np.isnan(x)]

        # 値の出現回数を数える
        value_counts = Counter(values)

        # 割合を計算し、値と割合を含むNumPyの2次元配列に格納
        result_array = np.array([(value, count / len(values) ) for value, count in value_counts.items()], dtype=float)

        # ソートしたい列のインデックスを指定（ここでは2列目をソートすると仮定）
        sort_column_index = 0

        try:
            # ソートのためのインデックスを取得
            sorted_indices = np.argsort(result_array[:, sort_column_index])[::-1]  # 降順にソート

            # 配列をソートしたインデックスに従って更新
            data = result_array[sorted_indices]

        except IndexError:
            # インデックスエラーが発生した場合、ここで処理をスキップ
            data = np.array([], dtype=int)  # 空のインデックス配列を作成

        return data

def pricing_for_next_day(data):
    array_list = []
    for i in range(len(data)):
        array_list.append(tree(data[i,0]))
    main_array = np.empty((0, 2))
    for j in range(len(array_list)):
        for k in range(len(array_list[j])):
            row = np.array([ array_list[j][k][0], data[j,1] * array_list[j][k][1]])
            main_array = np.vstack((main_array, row))
    # 結果を格納する新しいリストを作成
    result = []
    # ユニークな左の列の値を取得
    unique_left_column_values = np.unique(main_array[:, 0])
    # ユニークな左の列の値ごとに処理
    for value in unique_left_column_values:
        # 条件に合致する行を抽出
        rows_with_value = main_array[main_array[:, 0] == value]
        # 右の列の値を足し合わせて新しい行を作成し、結果リストに追加
        new_row = [value, np.sum(rows_with_value[:, 1])]
        result.append(new_row)
    # 結果をNumPy配列に変換
    result_array = np.array(result)
    # ソートしたい列のインデックスを指定（ここでは2列目をソートすると仮定）
    sort_column_index = 0
    # ソートのためのインデックスを取得
    sorted_indices = np.argsort(result_array[:, sort_column_index])[::-1]  # 降順にソート
    # 配列をソートしたインデックスに従って更新
    nextday = result_array[sorted_indices]
    # column_sum = np.sum(nextday[:, 1])  # 1番目の列を指定（0から始まるインデックスで1）
    # print("2列目の合計値:", column_sum)
    return nextday

# 出力用dataframe
df_price = pd.DataFrame(columns=['S_0', '2day', '3day', '7day', '14day'])
for x in unique_values_list:
    one = tree(x)
    price1 = max([ pricing(one, x), 0 ])
    two = pricing_for_next_day(one)
    price2 = max([ pricing(two, x), price1 ])
    three = pricing_for_next_day(two)
    price3 = max ([ pricing(three, x), price2 ])
    four = pricing_for_next_day(three)
    price4 = max ([ pricing(four, x), price3 ])
    five = pricing_for_next_day(four)
    price5 = max ([ pricing(five, x), price4 ])
    six = pricing_for_next_day(five)
    price6 = max ([ pricing(six, x), price5 ])
    seven = pricing_for_next_day(six)
    price7 = max ([ pricing(seven, x), price6 ])
    eight = pricing_for_next_day(seven)
    price8 = max ([ pricing(eight, x), price7 ])
    nine = pricing_for_next_day(eight)
    price9 = max ([ pricing(nine, x), price8 ])
    ten = pricing_for_next_day(nine)
    price10 = max ([ pricing(ten, x), price9 ])
    eleven = pricing_for_next_day(ten)
    price11 = max ([ pricing(eleven, x), price10 ])
    twelve = pricing_for_next_day(eleven)
    price12 = max ([ pricing(twelve, x), price11 ])
    thirteen = pricing_for_next_day(twelve)
    price13 = max ([ pricing(thirteen, x), price12 ])
    fourteen = pricing_for_next_day(thirteen)
    # print(fourteen)
    price14 = max ([ pricing(fourteen, x), price13 ])
    print(x,price2,price3,price7,price14)
    new_row = pd.DataFrame({'S_0': [x], '2day': [price2], '3day': [price3], '7day': [price7], '14day': [price14]})
    df_price = pd.concat([df_price, new_row], ignore_index=True)

168 41.6718178987926 62.0881033058834 116.344760244768 179.263056262667
173 18.3881424474814 29.8940622144593 74.0898005079300 136.872226066862


KeyboardInterrupt: 

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_price

,S_0,2day,3day,7day,14day
0,435.0,57.7318646091370,76.8506832039312,138.878753014203,210.226608131781
1,472.0,40.6394988631929,57.8281927892739,114.602850139478,180.313324248319
2,485.0,50.9657142871628,67.2494837318595,122.824141815367,184.279132410842
3,505.0,56.0407604741914,72.4894094254443,123.635925157765,178.114431499354
4,510.0,48.5512820497586,56.9536688383235,79.7951684683328,114.906710257923
5,522.0,32.9751000615254,48.0676378535725,98.8901718543863,154.116414463997
6,542.0,36.1999450617691,51.3915851249166,99.0843228507691,148.493925246379
7,547.0,21.4051262128426,27.9933677855687,49.1065532247174,83.5132759793430
8,554.0,65.3333333451905,91.1021367027191,160.209161116838,213.046307166134
9,555.0,69.3015872970845,89.4273075391460,130.802325884110,145.794420627244


In [30]:
# 数式表示
import sympy 
from sympy import *
import numpy as np
# gamma = 0.1
# 初期推定値を設定
initial_guess = 0.0  # 初期推定値
lower_bound = -300
upper_bound = 300
# 制約条件を定義
constraint = ({'type': 'ineq', 'fun': lambda beta: beta - lower_bound},  # beta >= lower_bound
            {'type': 'ineq', 'fun': lambda beta: upper_bound - beta})  # beta <= upper_bound

data = np.array([[90, 0.1], [100, 0.2], [110, 0.3], [130, 0.4]])
# data = tree(224)
target = 110

# 最小化したい関数を定義
def objective_function1(beta):
    term = 0
    for i in range(len(data)):
        if data[i,0] - target != 0:
            term += (1/(1-gamma)) * data[i,1] * (( data[i,0] - target ) * beta)**(1-gamma)
        else:
            term += 0
    return term
    # return np.exp(2 * beta) + np.exp(-2 * beta)

def objective_function2(beta):
    term = 0
    for i in range(len(data)):
        if data[i,0] != target:
            term += (1/(1-gamma)) * data[i,1] * (( data[i,0] - target ) * beta + max([ data[i,0] - target , 0 ]))**(1-gamma)
        else:
            term += 0
    return term
    # return math.exp(2 * beta) + math.exp(-2 * beta -2)

# # 最大化を実行
# result = minimize(objective_function1, initial_guess, constraints=constraint)
# # 最適な変数の値を表示
# beta1 = result.x[0]
# # print(beta1)
# # print(result.fun)

# # 最大化を実行
# result = minimize(objective_function2, initial_guess, constraints=constraint)
# # 最適な変数の値を表示
# beta2 = result.x[0]
# # print(beta2)
# # print(result.fun)

x = sympy.Symbol('x')
v = sympy.Symbol('v')
beta1 = sympy.Symbol('beta1')
beta2 = sympy.Symbol('beta2')
# gamma = sympy.Symbol('gamma')
gamma=0

A = 0
for i in range(len(data)):
    # A += - gamma * abs(data[i,1]) * ( x + ( data[i,0] - target ) * beta1 )**gamma
    A += - (1/(1-gamma)) * abs(data[i,1]) * ( x + ( data[i,0] - target ) * beta1 ) **(1-gamma)
B = 0
for i in range(len(data)):
    B += - (1/(1-gamma)) * abs(data[i,1]) * ( (x-v) + ( data[i,0] - target ) * beta2 + abs(max([ data[i,0] - target , 0 ])))**(1-gamma)


print(latex(simplify(A)))
print(latex(simplify(B)))
simplify(A)

sol = solve(A-B, v)
print(latex(sol[0]))
sol[0]

- 4.0 \beta_{1} - 1.0 x
- 4.0 \beta_{2} + 1.0 v - 1.0 x - 8.0
- 4.0 \beta_{1} + 4.0 \beta_{2} + 8.0


-4.0*beta1 + 4.0*beta2 + 8.0

In [6]:

# 翌日のツリーを作成
def tree(target_value):
    if np.any(np_array == target_value):
        # 条件に一致する位置を取得
        indices = np.where(np_array == target_value)

        # 一致する位置の行と列をndarrayに格納
        positions = np.array(list(zip(indices[0], indices[1])))

        # 列番号を1足し、その場所の値をリストに格納
        values = []
        for row, col in positions:
            if col != np_array.shape[1] - 1:
                values.append(np_array[row, col + 1])

        # nanを削除
        values = [x for x in values if not np.isnan(x)]

        # 値の出現回数を数える
        value_counts = Counter(values)

        # 割合を計算し、値と割合を含むNumPyの2次元配列に格納
        result_array = np.array([(value, count / len(values) ) for value, count in value_counts.items()], dtype=float)

        # ソートしたい列のインデックスを指定（ここでは2列目をソートすると仮定）
        sort_column_index = 0

        try:
            # ソートのためのインデックスを取得
            sorted_indices = np.argsort(result_array[:, sort_column_index])[::-1]  # 降順にソート

            # 配列をソートしたインデックスに従って更新
            data = result_array[sorted_indices]

        except IndexError:
            # インデックスエラーが発生した場合、ここで処理をスキップ
            data = np.array([], dtype=int)  # 空のインデックス配列を作成

        return data
tree(394)

array([[459.,   1.]])

In [1]:
values0 = [i * 0.1 for i in range(10, 1, -1)]
print(values0)

[1.0, 0.9, 0.8, 0.7000000000000001, 0.6000000000000001, 0.5, 0.4, 0.30000000000000004, 0.2]


In [10]:
import numpy as np

matrix = np.array([[1, 2, 3], [7, 5, 6]])

# 2行目の要素と10との差を計算
diff = np.abs(matrix[1] - 10)

# 最も差が小さい要素のインデックスを取得
index_of_closest = np.argmin(diff)

print("2行目の中で最も10に近い値のインデックス:", index_of_closest)
print(matrix[0,index_of_closest])

2行目の中で最も10に近い値のインデックス: 0
1
